# Basalt Observability End-to-End Playbook

This notebook demonstrates how to combine Basalt's observability helpers with common LLM flows, evaluators,
experiment tagging, and Google AI Studio (Gemini) interactions. Each scenario keeps telemetry metadata and
evaluators aligned with legacy monitor traces.

## Prerequisites

- Install the SDK in editable mode with dev extras: `uv pip install -e ".[dev]"`
- (Optional) Install the Google Generative AI SDK: `pip install google-generativeai`
- Set the following environment variables before running the notebook:
  - `BASALT_API_KEY` – your Basalt API key
  - `GOOGLE_API_KEY` – Google AI Studio key (Gemini)
  - `TRACELOOP_TRACE_CONTENT=1` if you want prompts/completions captured


In [37]:
import os
from datetime import datetime

from basalt import Basalt
from basalt.observability import (
    add_default_evaluators,
    attach_trace_experiment,
    configure_trace_defaults,
    trace_event,
    trace_llm_call,
    trace_retrieval,
    trace_span,
    trace_tool,
)
from basalt.observability.decorators import trace_llm, trace_operation
from basalt.observability.trace_context import TraceContextConfig

try:
    from google import genai
except ImportError:  # pragma: no cover - optional dependency
    genai = None


## 1. Configure the Basalt client and default trace context

We prime global defaults so that every span carries user, organization, and evaluator metadata.


In [38]:
# Configure defaults before instantiating the client
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter

from basalt.observability.config import TelemetryConfig

configure_trace_defaults(
    user={"id": "user-notebook", "name": "Analyst"},
    organization={"id": "org-research", "name": "Basalt Research"},
    metadata={"environment": "notebook", "workspace": "demo"},
    evaluators=["accuracy"],
)
add_default_evaluators("toxicity")
exporter = OTLPSpanExporter(endpoint="http://127.0.0.1:4317", insecure=True)
telemetry = TelemetryConfig(service_name="notebook", exporter=exporter)

basalt_client = Basalt(
    api_key=os.getenv("BASALT_API_KEY", "not-set"),
    trace_experiment={"id": "exp-observability", "feature_slug": "demo-agent"},
    trace_metadata={"notebook": "observability-playbook"},
    telemetry_config=telemetry,

)

basalt_client


## 2. Decorator-driven LLM spans with evaluators

The `@trace_llm` decorator grabs prompts, completions, and token usage automatically.


In [ ]:
@trace_llm(name="notebook.gemini.summarize")
def summarize_with_gemini(prompt: str, *, model: str = "gemini-2.5-flash-lite") -> str | None:
    if genai is None:
        raise RuntimeError("google-generativeai is not installed")

    client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY", "fake-key"))
    response = client.models.generate_content(model=model, contents=prompt)
    # Convert response to dict so the decorator can introspect usage
    return response.text

try:
    gemini_result = summarize_with_gemini("Summarize the benefits of synthetic monitoring.")
except Exception as exc:
    gemini_result = {"error": str(exc)}

gemini_result


"Synthetic monitoring offers a proactive and powerful way to ensure the performance, availability, and reliability of your applications and digital services. Here's a summary of its key benefits:\n\n**1. Proactive Problem Detection (Early Warning System):**\n* **Detects issues before users do:** By simulating user interactions, synthetic monitoring identifies problems before they impact real customers, allowing for faster resolution and preventing customer frustration.\n* **Reduces downtime:** Catching issues early minimizes the duration of outages, protecting revenue, brand reputation, and user trust.\n\n**2. Continuous Availability and Performance Assurance:**\n* **Constant vigilance:** Synthetic checks run around the clock, providing continuous assurance that your application is up and running and performing as expected.\n* **Baseline establishment:** It helps establish a baseline for normal performance, making it easier to identify deviations and anomalies.\n\n**3. Performance Opti

## 3. Manual spans for orchestrating workflow stages

We combine `trace_span`, `trace_tool`, and `trace_event` to follow a retrieval-augmented generation pipeline.


In [40]:
with trace_span("workflow.rag", attributes={"feature": "support-bot"}) as span:
    span.add_evaluator("latency-budget")
    span.set_experiment("exp-rag-001", feature_slug="support-bot")

    with trace_retrieval("workflow.rag.retrieve") as ret_span:
        ret_span.set_query("error connecting to database")
        ret_span.set_results_count(3)
        ret_span.set_top_k(5)

    with trace_tool("workflow.rag.tool") as tool_span:
        tool_span.set_tool_name("web-search")
        tool_span.set_input({"query": "database connection refused troubleshooting"})
        tool_span.set_output({"summary": "Check credentials and firewall rules."})

    with trace_llm_call("workflow.rag.answer") as llm_span:
        llm_span.set_model("gemini-1.5-flash")
        llm_span.set_prompt("Provide mitigation steps")
        llm_span.set_completion("1. Verify credentials...")
        llm_span.set_tokens(input=200, output=180)

    with trace_event("workflow.rag.event") as event_span:
        event_span.set_event_type("handoff")
        event_span.set_payload({"team": "support", "status": "ready"})


## 4. Experiments and trace enrichment APIs

Attach experiment metadata globally, then override within the active span when needed.


In [30]:
attach_trace_experiment("exp-baseline", name="baseline-playbook", feature_slug="demo-agent")

with trace_span("workflow.ab-test") as span:
    span.set_experiment("exp-variant", name="variant-b", feature_slug="demo-agent-b")
    span.add_evaluator("judge-hallucination")
    span.add_event("scoring_started", {"timestamp": datetime.utcnow().isoformat()})
    span.set_attribute("basalt.metric.latency_ms", 245)


/tmp/ipykernel_106982/287295438.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  span.add_event("scoring_started", {"timestamp": datetime.utcnow().isoformat()})


## 5. Shutdown and cleanup

Flush telemetry buffers when the workflow completes.

In [31]:
basalt_client.shutdown()
